In [3]:
words = open('names.txt', 'r').read().splitlines()

E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?

In [2]:
import torch
import torch.nn.functional as F

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [28]:
# bigram
xs_b, ys_b = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        xs_b.append(stoi[ch1])
        ys_b.append(stoi[ch2])
        
xs_b = torch.tensor(xs_b)
ys_b = torch.tensor(ys_b)
num = xs_b.shape[0]
print('number of examples:', num)

number of examples: 228146


In [33]:
# trigram
xs_t, ys_t = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        xs_t.append([stoi[ch1], stoi[ch2]])
        ys_t.append(stoi[ch3])
        
xs_t = torch.tensor(xs_t)
ys_t = torch.tensor(ys_t)
num = xs_t.shape[0]
print('number of examples:', num)

number of examples: 196113


In [14]:
# initialize the model parameters
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27*27, 27), generator=g, requires_grad=True)

In [35]:
# gradient descent
for k in range(10):
    # forward pass
    bigram_indices = xs_t[:, 0] * 27 + xs_t[:, 1]  # Convert bigram (ch1, ch2) into a single index
    xenc = F.one_hot(bigram_indices, 27*27).float() # one-hot encoding
    
    logits = (xenc @ W) # log-counts
    counts = logits.exp() # equivalent to N
    probs = counts / counts.sum(1, keepdim=True) # probabilities for the next character
    loss = -probs[torch.arange(len(ys_t)), ys_t].log().mean() + 0.01*(W**2).mean() # negative log-likelihood
    print(loss.item())
    
    # backward pass
    W.grad = None # set the gradients to zero
    loss.backward() # backpropagate
    
    # update
    W.data += -50 * W.grad # update the parameters

2.409698009490967
2.4073660373687744
2.405068874359131
2.4028050899505615
2.400573492050171
2.39837384223938
2.3962056636810303
2.3940680027008057
2.391960382461548
2.3898818492889404


In [ ]:
bigram_indices = xs_t[:, 0] * 27 + xs_t[:, 1]  # Convert bigram (ch1, ch2) into a single index
xenc = F.one_hot(bigram_indices, 27*27).float() # one-hot encoding
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
probs

tensor([[0.0098, 0.0209, 0.0125,  ..., 0.0064, 0.0027, 0.0299],
        [0.1306, 0.1681, 0.0358,  ..., 0.0118, 0.0309, 0.0058],
        [0.0609, 0.1708, 0.0103,  ..., 0.0038, 0.0987, 0.0122],
        ...,
        [0.1185, 0.0283, 0.0108,  ..., 0.0135, 0.0538, 0.0141],
        [0.0071, 0.0465, 0.0601,  ..., 0.1148, 0.0740, 0.0932],
        [0.0116, 0.0305, 0.0147,  ..., 0.0123, 0.0027, 0.0175]],
       grad_fn=<DivBackward0>)


In [38]:
probs.shape

torch.Size([196113, 27])

In [49]:
import random

# Define a function for sampling from the trained model
def sample_trigram_model(W, stoi, itos, max_length=20, seed_chars=['.'], g=torch.Generator()):
    """
    Generate a word using the trained trigram model.

    Parameters:
    - W: Trained weight matrix (729, 27)
    - stoi: Dictionary mapping characters to indices
    - itos: Dictionary mapping indices back to characters
    - max_length: Maximum length of the generated sequence
    - seed_chars: Starting character(s) for generation

    Returns:
    - Generated sequence as a string
    """
    out = seed_chars[:]  # Initialize output with the seed characters

    # Convert initial seed characters to indices
    if len(seed_chars) == 1:
        seed_chars.append(random.choice(list(stoi.keys())))  # Append a random character

    ch1, ch2 = seed_chars[-2], seed_chars[-1]  # Start with the last two characters
    
    for _ in range(max_length):
        # Convert (ch1, ch2) into a trigram index
        trigram_index = stoi[ch1] * 27 + stoi[ch2]  # Encoding (bigram → single index)
        xenc = F.one_hot(torch.tensor(trigram_index), num_classes=27 * 27).float()

        # Forward pass: Compute probabilities for the next character
        logits = xenc @ W
        counts = logits.exp()
        probs = counts / counts.sum()  # Normalize probabilities

        # Sample the next character
        ix = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])

        # Stop if the end token (".") is generated
        if ix == 0:
            break

        # Shift context for next trigram prediction
        ch1, ch2 = ch2, itos[ix]

    return ''.join(out)

# Example usage
generated_word = sample_trigram_model(W, stoi, itos)
print(f"Generated word: {generated_word}")


Generated word: .cnbiltexfhfglyn.


In [52]:
for _ in range(10):
    generated_word = sample_trigram_model(W, stoi, itos, seed_chars=['t', 'a'], g=g)
    print(f"Generated word: {generated_word}")

Generated word: tah.
Generated word: talithaedhtedarlnhrqbk
Generated word: tala.
Generated word: taglyan.
Generated word: tayzkenpnsomspigogd.
Generated word: tatavaniufoczlzduokowh
Generated word: tarwdszskgroson.
Generated word: taqmgmerlfddpqxplynuim
Generated word: talie.
Generated word: takwvhynnxbbzdotj.


E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?

In [29]:
from torch.utils.data import random_split

def split_data(xs, ys, train_ratio=0.8, dev_ratio=0.1, test_ratio=0.1, seed=2147483647):
    num_samples = xs.shape[0]
    num_train = int(num_samples * train_ratio)
    num_dev = int(num_samples * dev_ratio)
    num_test = num_samples - num_train - num_dev

    # shuffle before splitting
    indices = torch.randperm(num_samples, generator=g)

    xs_shuffled = xs[indices]
    ys_shuffled = ys[indices]

    # split the dataset
    xs_train, xs_dev, xs_test = torch.split(xs_shuffled, [num_train, num_dev, num_test])
    ys_train, ys_dev, ys_test = torch.split(ys_shuffled, [num_train, num_dev, num_test])
    
    return xs_train, xs_dev, xs_test, ys_train, ys_dev, ys_test

In [68]:
xs_train_b, xs_dev_b, xs_test_b, ys_train_b, ys_dev_b, ys_test_b = split_data(xs_b, ys_b)
xs_train_t, xs_dev_t, xs_test_t, ys_train_t, ys_dev_t, ys_test_t = split_data(xs_t, ys_t)

In [59]:
# initialize the model parameters
g = torch.Generator().manual_seed(2147483647)
W_t = torch.randn((27*27, 27), generator=g, requires_grad=True)
W_b = torch.randn((27, 27), generator=g, requires_grad=True)

In [63]:
# gradient descent bigram
for k in range(10):
    # forward pass
    xenc = F.one_hot(xs_train_b, 27).float() # one-hot encoding
    logits = (xenc @ W_b) # log-counts
    counts = logits.exp() # equivalent to N
    probs = counts / counts.sum(1, keepdim=True) # probabilities for the next character
    loss = -probs[torch.arange(len(ys_train_b)), ys_train_b].log().mean() + 0.01*(W_b**2).mean() # negative log-likelihood
    print(loss.item())
    
    # backward pass
    W_b.grad = None # set the gradients to zero
    loss.backward() # backpropagate
    
    # update
    W_b.data += -100 * W_b.grad # update the parameters

2.5480940341949463
2.5636048316955566
2.5408473014831543
2.5470118522644043
2.539839267730713
2.5565619468688965
2.5335073471069336
2.5397894382476807
2.5338873863220215
2.551547050476074


In [72]:
# gradient descent trigram
for k in range(10):
    # forward pass
    bigram_indices = xs_train_t[:, 0] * 27 + xs_train_t[:, 1]  # Convert bigram (ch1, ch2) into a single index
    xenc = F.one_hot(bigram_indices, 27*27).float() # one-hot encoding
    
    logits = (xenc @ W_t) # log-counts
    counts = logits.exp() # equivalent to N
    probs = counts / counts.sum(1, keepdim=True) # probabilities for the next character
    loss = -probs[torch.arange(len(ys_train_t)), ys_train_t].log().mean() + 0.01*(W_t**2).mean() # negative log-likelihood
    print(loss.item())
    
    # backward pass
    W_t.grad = None # set the gradients to zero
    loss.backward() # backpropagate
    
    # update
    W_t.data += -100 * W_t.grad # update the parameters

2.9147050380706787
2.878364086151123
2.845487117767334
2.8156321048736572
2.788423538208008
2.7635366916656494
2.7406864166259766
2.7196226119995117
2.7001261711120605
2.6820075511932373


In [74]:
# bigram dev & test loss
xenc = F.one_hot(xs_dev_b, 27).float() # one-hot encoding
logits = (xenc @ W_b) # log-counts
counts = logits.exp() # equivalent to N
probs = counts / counts.sum(1, keepdim=True) # probabilities for the next character
loss = -probs[torch.arange(len(ys_dev_b)), ys_dev_b].log().mean() + 0.01*(W_b**2).mean() # negative log-likelihood
print(loss.item())

xenc = F.one_hot(xs_test_b, 27).float() # one-hot encoding
logits = (xenc @ W_b) # log-counts
counts = logits.exp() # equivalent to N
probs = counts / counts.sum(1, keepdim=True) # probabilities for the next character
loss = -probs[torch.arange(len(ys_test_b)), ys_test_b].log().mean() + 0.01*(W_b**2).mean() # negative log-likelihood
print(loss.item())

2.545044422149658
2.5196807384490967


In [75]:
# trigram dev & test loss
bigram_indices = xs_dev_t[:, 0] * 27 + xs_dev_t[:, 1]  # Convert bigram (ch1, ch2) into a single index
xenc = F.one_hot(bigram_indices, 27*27).float() # one-hot encoding

logits = (xenc @ W_t) # log-counts
counts = logits.exp() # equivalent to N
probs = counts / counts.sum(1, keepdim=True) # probabilities for the next character
loss = -probs[torch.arange(len(ys_dev_t)), ys_dev_t].log().mean() + 0.01*(W_t**2).mean() # negative log-likelihood
print(loss.item())

bigram_indices = xs_test_t[:, 0] * 27 + xs_test_t[:, 1]  # Convert bigram (ch1, ch2) into a single index
xenc = F.one_hot(bigram_indices, 27*27).float() # one-hot encoding

logits = (xenc @ W_t) # log-counts
counts = logits.exp() # equivalent to N
probs = counts / counts.sum(1, keepdim=True) # probabilities for the next character
loss = -probs[torch.arange(len(ys_test_t)), ys_test_t].log().mean() + 0.01*(W_t**2).mean() # negative log-likelihood
print(loss.item())

2.6630184650421143
2.682529926300049


E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?

In [98]:
def train_trigram_model(xs_train, ys_train, xs_dev, ys_dev, W_t, reg_strength, num_epochs=100, lr=10):
    """
    Train the trigram model for a given regularization strength (smoothing).
    """
    losses = {"train": [], "dev": []}  # Store losses for analysis
    
    for k in range(num_epochs):
        # Forward pass
        bigram_indices = xs_train[:, 0] * 27 + xs_train[:, 1]  # Convert bigram (ch1, ch2) into a single index
        logits = W_t[bigram_indices]
        counts = logits.exp()  # Equivalent to N
        probs = counts / counts.sum(1, keepdim=True)  # Probabilities for the next character
        loss_train = -probs[torch.arange(len(ys_train)), ys_train].log().mean() + reg_strength * (W_t**2).mean()  # Negative log-likelihood
        losses["train"].append(loss_train.item())
        
        # Backward pass
        W_t.grad = None  # Set the gradients to zero
        loss_train.backward()  # Backpropagate
        with torch.no_grad():
            W_t -= lr * W_t.grad
            
        # Compute dev loss
        with torch.no_grad():
            bigram_indices_dev = xs_dev[:, 0] * 27 + xs_dev[:, 1]
            logits_dev = W_t[bigram_indices_dev]
            counts_dev = logits_dev.exp()
            probs_dev = counts_dev / counts_dev.sum(1, keepdim=True)
            loss_dev = -probs_dev[torch.arange(len(ys_dev)), ys_dev].log().mean() + reg_strength * (W_t**2).mean()
            losses["dev"].append(loss_dev.item())
            
        # Print loss
        print(f"λ={reg_strength}, Epoch {k}: Train Loss = {loss_train.item():.4f}, Dev Loss = {loss_dev.item():.4f}")
        
    return W_t, losses

In [99]:
# Try different regularization strengths
reg_strengths = [0.001, 0.01, 0.1, 1, 10]  # Different lambda values
best_loss = float('inf')
best_W_t = None
best_lambda = None
all_losses = {}

for reg_strength in reg_strengths:
    print(f"Training with λ={reg_strength}")
    
    # Initialize the model parameters
    W_t = torch.randn((27*27, 27), generator=torch.Generator().manual_seed(2147483647), requires_grad=True)
    
    # Train model
    W_t, losses = train_trigram_model(xs_train_t, ys_train_t, xs_dev_t, ys_dev_t, W_t, reg_strength)
    all_losses[reg_strength] = losses
    
    # Check if the current model is the best
    if losses["dev"][-1] < best_loss:
        best_loss = losses["dev"][-1]
        best_W_t = W_t
        best_lambda = reg_strength
        


Training with λ=0.001
λ=0.001, Epoch 0: Train Loss = 3.7248, Dev Loss = 3.7042
λ=0.001, Epoch 1: Train Loss = 3.7095, Dev Loss = 3.6890


λ=0.001, Epoch 2: Train Loss = 3.6943, Dev Loss = 3.6740
λ=0.001, Epoch 3: Train Loss = 3.6793, Dev Loss = 3.6592
λ=0.001, Epoch 4: Train Loss = 3.6645, Dev Loss = 3.6445
λ=0.001, Epoch 5: Train Loss = 3.6499, Dev Loss = 3.6301
λ=0.001, Epoch 6: Train Loss = 3.6355, Dev Loss = 3.6158
λ=0.001, Epoch 7: Train Loss = 3.6212, Dev Loss = 3.6018
λ=0.001, Epoch 8: Train Loss = 3.6072, Dev Loss = 3.5879
λ=0.001, Epoch 9: Train Loss = 3.5933, Dev Loss = 3.5742
λ=0.001, Epoch 10: Train Loss = 3.5796, Dev Loss = 3.5607
λ=0.001, Epoch 11: Train Loss = 3.5661, Dev Loss = 3.5473
λ=0.001, Epoch 12: Train Loss = 3.5528, Dev Loss = 3.5342
λ=0.001, Epoch 13: Train Loss = 3.5396, Dev Loss = 3.5212
λ=0.001, Epoch 14: Train Loss = 3.5267, Dev Loss = 3.5084
λ=0.001, Epoch 15: Train Loss = 3.5139, Dev Loss = 3.4958
λ=0.001, Epoch 16: Train Loss = 3.5013, Dev Loss = 3.4834
λ=0.001, Epoch 17: Train Loss = 3.4889, Dev Loss = 3.4712
λ=0.001, Epoch 18: Train Loss = 3.4767, Dev Loss = 3.4591
λ=0.001, Epoch 19: Tra

In [100]:
# Print best λ found
print(f"\nBest regularization strength: λ = {best_lambda} with Dev Loss = {best_loss:.4f}")


Best regularization strength: λ = 0.001 with Dev Loss = 2.9084


In [101]:
# Evaluate best model on test set
with torch.no_grad():
    bigram_indices_test = xs_test_t[:, 0] * 27 + xs_test_t[:, 1]
    logits_test = W_t[bigram_indices_test]
    counts_test = logits_test.exp()
    probs_test = counts_test / counts_test.sum(1, keepdim=True)

    loss_test = -probs_test[torch.arange(len(ys_test_t)), ys_test_t].log().mean() + best_lambda * (best_W_t**2).mean()

print(f"\nFinal Test Loss with best λ={best_lambda}: {loss_test:.4f}")


Final Test Loss with best λ=0.001: 2.8541


E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?

In [105]:
# Evaluate best model on test set
with torch.no_grad():
    bigram_indices_test = xs_test_t[:, 0] * 27 + xs_test_t[:, 1]
    logits_test = best_W_t[bigram_indices_test]
    counts_test = logits_test.exp()
    probs_test = counts_test / counts_test.sum(1, keepdim=True)

    loss_test = -probs_test[torch.arange(len(ys_test_t)), ys_test_t].log().mean() + best_lambda * (best_W_t**2).mean()

print(f"\nFinal Test Loss with best λ={best_lambda}: {loss_test:.4f}")


Final Test Loss with best λ=0.001: 2.9254


In [106]:
# bigram dev & test loss
logits = W_b[xs_dev_b]
counts = logits.exp() # equivalent to N
probs = counts / counts.sum(1, keepdim=True) # probabilities for the next character
loss = -probs[torch.arange(len(ys_dev_b)), ys_dev_b].log().mean() + 0.01*(W_b**2).mean() # negative log-likelihood
print(loss.item())

logits = W_b[xs_test_b]
counts = logits.exp() # equivalent to N
probs = counts / counts.sum(1, keepdim=True) # probabilities for the next character
loss = -probs[torch.arange(len(ys_test_b)), ys_test_b].log().mean() + 0.01*(W_b**2).mean() # negative log-likelihood
print(loss.item())

2.545044422149658
2.5196807384490967


E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?

In [110]:
# Evaluate best model on test set
with torch.no_grad():
    bigram_indices_test = xs_test_t[:, 0] * 27 + xs_test_t[:, 1]
    logits_test = best_W_t[bigram_indices_test]
    counts_test = logits_test.exp()
    probs_test = counts_test / counts_test.sum(1, keepdim=True)

    loss_test = F.cross_entropy(logits_test, ys_test_t) + best_lambda * (best_W_t**2).mean()

print(f"\nFinal Test Loss with best λ={best_lambda}: {loss_test:.4f}")


Final Test Loss with best λ=0.001: 2.9254
